In [1]:
import pandas as pd
import sys
sys.path.append('../')
from common import load_clean_train

In [3]:
train_df = load_clean_train()
TRAIN_STREETS = train_df['Street'].unique()
TRAIN_STREETS = [x.upper().replace('STREET', 'ST').replace('AVENUE', 'AVE') for x in TRAIN_STREETS]
TRAIN_STREETS

### Trimming down data

In [24]:
sn_df = pd.read_feather('/Users/timlee/data/sf_parking/P_sensor.feather')
ORIG_SENSOR_STREETS = sn_df['STREET_NAME'].unique()
sn_df = sn_df[sn_df['STREET_NAME'].isin(TRAIN_STREETS)]
dt_str = sn_df['START_TIME_DT'].unique()
lkup = {str_v : pd.to_datetime(str_v) for str_v in dt_str}

sn_df['START_TIME_DT'] = sn_df['START_TIME_DT'].map(lambda x : lkup[x])
sn_df['tot_spaces'] = sn_df['TOTAL_TIME'] / 3600.
sn_df['occ_spaces'] = sn_df['TOTAL_OCCUPIED_TIME'] / 3600.
sn_df['vac_spaces'] = sn_df['TOTAL_VACANT_TIME'] / 3600.
sn_df['unk_spaces'] = sn_df['TOTAL_UNKNOWN_TIME'] / 3600.
sn_df['occ_spaces_pct'] = sn_df['occ_spaces']/ sn_df['tot_spaces']
sn_df['vac_spaces_pct'] = sn_df['vac_spaces'] / sn_df['tot_spaces']
sn_df['unk_spaces_pct'] = sn_df['unk_spaces'] / sn_df['tot_spaces']
sn_df['day'] = sn_df['START_TIME_DT'].dt.day
sn_df['mon'] = sn_df['START_TIME_DT'].dt.month
sn_df['dow'] = sn_df['START_TIME_DT'].dt.dayofweek
sn_df['hour'] = sn_df['START_TIME_DT'].dt.hour
sn_df.reset_index(inplace=True)
print(sn_df.shape)
sn_df.to_feather('/Users/timlee/data/sf_parking/sensor_small.feather')

(1911247, 45)


In [25]:
ORIG_SENSOR_STREETS

array(['FILLMORE ST', 'JACKSON ST', '09TH AVE', '10TH AVE', '14TH AVE',
       'BERRY ST', '01ST ST', 'CALIFORNIA ST', 'CHESTNUT ST', 'FOLSOM ST',
       'JONES ST', '02ND ST', 'FRANKLIN ST', 'MARKET ST', '03RD ST',
       'CLAY ST', 'FREMONT ST', 'FRONT ST', '04TH ST', 'GEARY BLVD',
       'MASON ST', 'CLEMENT ST', 'KEARNY ST', '18TH ST', '22ND ST',
       'LEAVENWORTH ST', '23RD ST', 'BATTERY ST', 'DAVIS ST',
       'GOLDEN GATE AVE', 'MCALLISTER ST', 'BAY ST', 'DIVISADERO ST',
       'DRUMM ST', 'GOUGH ST', 'GROVE ST', 'LAGUNA ST', 'BEACH ST',
       'FELL ST', 'GREENWICH ST', 'HAWTHORNE ST', 'LOMBARD ST',
       'MALLORCA WAY', 'BEALE ST', 'BRANNAN ST', 'HAYES ST', 'BRYANT ST',
       'HOWARD ST', 'FILBERT ST', 'HOFF ST', 'BUCHANAN ST', 'BUSH ST',
       'JESSIE ST', 'MONTGOMERY ST', 'LARKIN ST', 'KING ST',
       'COLUMBUS AVE', 'LECH WALESA ST', 'AVILA ST', 'CAPP ST',
       'MISSION ST', 'HICKORY ST', 'NORTH POINT ST', 'OCTAVIA ST',
       'NATOMA ST', 'MERCHANT ST', 'PIXLEY ST'

In [20]:
SENSOR_STREETS = sn_df['STREET_NAME'].unique()
SENSOR_STREETS

array(['JACKSON ST', 'JONES ST', 'MASON ST', 'KEARNY ST', 'LEAVENWORTH ST',
       '23RD ST', 'BATTERY ST', 'GROVE ST', 'BRYANT ST', 'BUSH ST',
       'JESSIE ST', 'MONTGOMERY ST', 'LARKIN ST', 'COLUMBUS AVE',
       'MISSION ST', 'POLK ST', 'TAYLOR ST', 'REDWOOD ST', 'WASHINGTON ST',
       'POST ST', 'PINE ST', 'HYDE ST', 'STOCKTON ST', 'SUTTER ST',
       'VAN NESS AVE'], dtype=object)

### Will have to make GEARY like another street

In [19]:
[x for x in TRAIN_STREETS if x not in SENSOR_STREETS]

['GEARY ST']

In [23]:
bbb = sn_df[sn_df['STREET_NAME']=='JACKSON ST'].groupby(['dow','hour'])['vac_spaces'].mean().reset_index()
bbb.pivot(index='hour', columns = 'dow', values = 'vac_spaces').reset_index()

dow,hour,0,1,2,3,4,5,6
0,0,9.469629,9.780590,9.730809,9.352232,9.304764,4.908751,2.569731
1,1,9.719062,10.099841,10.022693,9.800505,9.962757,7.058030,4.617688
2,2,9.778209,10.044318,9.965030,9.816986,10.018745,9.305049,8.272451
3,3,9.720063,9.913666,9.871797,9.723991,9.949473,9.986848,9.243064
4,4,9.786826,9.929340,9.956642,9.766241,10.019343,10.140577,9.326377
5,5,9.378218,9.564155,9.586384,9.382872,9.632069,10.088864,9.193093
6,6,8.007075,8.094056,8.032303,7.964834,8.067944,9.758869,8.920662
7,7,7.111954,6.759990,6.361300,6.550607,6.887343,8.719646,8.228231
8,8,6.187426,5.694843,5.302425,5.581166,5.750814,7.402779,5.961365
9,9,5.201196,4.457783,4.403234,4.624955,4.713714,7.042205,3.000744


In [8]:
sn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7902291 entries, 0 to 7902290
Data columns (total 44 columns):
BLOCK_ID               object
STREET_NAME            object
BLOCK_NUM              float64
STREET_BLOCK           object
AREA_TYPE              object
PM_DISTRICT_NAME       object
RATE                   float64
START_TIME_DT          datetime64[ns]
TOTAL_TIME             float64
TOTAL_OCCUPIED_TIME    float64
TOTAL_VACANT_TIME      float64
TOTAL_UNKNOWN_TIME     float64
OP_TIME                float64
OP_OCCUPIED_TIME       float64
OP_VACANT_TIME         float64
OP_UNKNOWN_TIME        float64
NONOP_TIME             float64
NONOP_OCCUPIED_TIME    float64
NONOP_VACANT_TIME      float64
NONOP_UNKNOWN_TIME     float64
GMP_TIME               float64
GMP_OCCUPIED_TIME      float64
GMP_VACANT_TIME        float64
GMP_UNKNOWN_TIME       float64
COMM_TIME              float64
COMM_OCCUPIED_TIME     float64
COMM_VACANT_TIME       float64
COMM_UNKNOWN_TIME      float64
CAL_MONTH_NAME   